In [1]:
# ==================================================
# 0. IMPORT LIBRARIES
# ==================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, mean_absolute_error, mean_squared_error,
    roc_auc_score
)

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [3]:
# ==================================================
# 1. LOAD DATASET
# ==================================================
file_path = r"E:\WOU_BITS INTERN\39 BUS SYSTEM\39 BUS SYSTEM FAULT.xlsx"
df = pd.read_excel(file_path)

print("Original Dataset Shape:", df.shape)

Original Dataset Shape: (135136, 24)


In [4]:
# ==================================================
# 2. FILTER ONLY FAULT LINE 1_39
# ==================================================
df = df[df["Fault Line"] == "1_39"].copy()

print("Filtered Dataset Shape (Only 1_39):", df.shape)
print("Available Fault Lines:", df["Fault Line"].unique())

Filtered Dataset Shape (Only 1_39): (4004, 24)
Available Fault Lines: ['1_39']


In [51]:
# ==================================================
# 3. DEFINE FEATURES AND LABELS
# ==================================================
FEATURES = [
    "Phase Voltage A/Terminal in kV",
    "Phase Voltage B/Terminal in kV",
    "Phase Voltage C/Terminal in kV",
    "Phase Current A/Terminal in kA",
    "Phase Current B/Terminal in kA",
    "Phase Current C/Terminal in kA",
    "Terminal/Bus"
]

# 🔥 Removed "Fault Line" because it is fixed now
LABELS = [
    "Fault Location",
    "Primary",
    "CCT",
    "B1"
]

X = df[FEATURES]

In [53]:
# ==================================================
# 4. FEATURE SCALING
# ==================================================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [11]:
# ==================================================
# 5. FINAL RESULT STORAGE
# ==================================================
final_summary = {}

In [55]:
# ==================================================
# 6. LOOP FOR EACH LABEL
# ==================================================
for label in LABELS:

    print("\n" + "=" * 60)
    print(f"🎯 LABEL: {label}")
    print("=" * 60)

    y = LabelEncoder().fit_transform(df[label])
    n_classes = len(np.unique(y))

    if n_classes < 2:
        print("❌ Only one class present — skipped")
        continue

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y,
        test_size=0.25,
        random_state=42,
        stratify=y
    )


🎯 LABEL: Fault Location

🎯 LABEL: Primary

🎯 LABEL: CCT

🎯 LABEL: B1


In [57]:
    # ==================================================
    # 7. RANDOM FOREST
    # ==================================================
    rf = RandomForestClassifier(
        n_estimators=300,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X_train, y_train)
    rf_prob = rf.predict_proba(X_test)
    rf_pred = np.argmax(rf_prob, axis=1)

    acc_rf = accuracy_score(y_test, rf_pred) * 100

In [61]:
# ==================================================
# XGBOOST (Binary Classification Version)
# ==================================================

if len(np.unique(y_train)) < 2:
    print("⚠️ XGBoost skipped — only one class in training split")
    acc_xgb = 0
    xgb_prob = None
    xgb_pred = None
else:
    xgb = XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary:logistic",   # ✅ Binary objective
        eval_metric="logloss",
        use_label_encoder=False,
        random_state=42,
        n_jobs=-1
    )

    xgb.fit(X_train, y_train)

    # Prediction
    xgb_pred = xgb.predict(X_test)
    xgb_prob = xgb.predict_proba(X_test)

    acc_xgb = accuracy_score(y_test, xgb_pred) * 100

C:\Users\ojash\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [01:36:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [63]:
    # ==================================================
    # ENSEMBLE (Auto Alpha Search)
    # ==================================================
    best_alpha = 0
    best_acc = 0
    best_pred = None
    best_prob = None

    for alpha in np.arange(0.1, 1.0, 0.1):
        ens_prob = alpha * rf_prob + (1 - alpha) * xgb_prob
        ens_pred = np.argmax(ens_prob, axis=1)
        acc = accuracy_score(y_test, ens_pred) * 100

        if acc > best_acc:
            best_acc = acc
            best_alpha = alpha
            best_pred = ens_pred
            best_prob = ens_prob

In [65]:
    # ==================================================
    # 10. MODEL SELECTION
    # ==================================================
    scores = {
        "Random Forest": acc_rf,
        "XGBoost": acc_xgb,
        "RF + XGB Ensemble": best_ens_acc
    }

    best_model = max(scores, key=scores.get)

In [67]:
    # ==================================================
    # FINAL METRICS
    # ==================================================
    acc = accuracy_score(y_test, y_pred) * 100
    prec = precision_score(y_test, y_pred, average="weighted") * 100
    rec = recall_score(y_test, y_pred, average="weighted") * 100
    f1 = f1_score(y_test, y_pred, average="weighted") * 100
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    try:
        roc_auc = roc_auc_score(y_test, y_prob, multi_class="ovr") * 100
    except:
        roc_auc = None

In [69]:
    # ==================================================
    # PRINT RESULTS
    # ==================================================
    print(f"RF Accuracy       : {acc_rf:.2f}%")
    print(f"XGB Accuracy      : {acc_xgb:.2f}%")
    print(f"Ensemble Accuracy : {best_acc:.2f}% (alpha={best_alpha})")
    print(f"✅ Selected Model  : {best_model}")

    print("\n--- FINAL METRICS ---")
    print(f"Accuracy  : {acc:.2f}%")
    print(f"Precision : {prec:.2f}%")
    print(f"Recall    : {rec:.2f}%")
    print(f"F1-score  : {f1:.2f}%")
    print(f"ROC-AUC   : {roc_auc:.2f}%" if roc_auc else "ROC-AUC : Not defined")
    print(f"MAE       : {mae:.4f}")
    print(f"MSE       : {mse:.4f}")
    print(f"RMSE      : {rmse:.4f}")

    final_summary[label] = {
        "Best Model": best_model,
        "Accuracy": acc
    }

RF Accuracy       : 100.00%
XGB Accuracy      : 100.00%
Ensemble Accuracy : 100.00% (alpha=0.1)
✅ Selected Model  : Random Forest

--- FINAL METRICS ---
Accuracy  : 100.00%
Precision : 100.00%
Recall    : 100.00%
F1-score  : 100.00%
ROC-AUC : Not defined
MAE       : 0.0000
MSE       : 0.0000
RMSE      : 0.0000


In [73]:
# ==================================================
# FINAL SUMMARY
# ==================================================
print("\n🎯 FINAL SUMMARY (Fault Line 1_39)")
print("=" * 60)

for lbl, res in final_summary.items():
    print(f"{lbl:15s} → {res['Best Model']:15s} | Accuracy = {res['Accuracy']:.2f}%")

print("\n✅ PROCESS COMPLETED SUCCESSFULLY")


🎯 FINAL SUMMARY (Fault Line 1_39)
B4              → Random Forest   | Accuracy = 100.00%
B1              → Random Forest   | Accuracy = 100.00%

✅ PROCESS COMPLETED SUCCESSFULLY


In [49]:
print("\nClass Distribution After Filtering:\n")
for col in ["Fault Location", "Primary", "CCT", "B1", "B4"]:
    print(col)
    print(df[col].value_counts())
    print("-" * 40)


Class Distribution After Filtering:

Fault Location
Fault Location
1     2002
99    2002
Name: count, dtype: int64
----------------------------------------
Primary
Primary
R2     2002
R36    2002
Name: count, dtype: int64
----------------------------------------
CCT
CCT
2.288    2002
2.432    2002
Name: count, dtype: int64
----------------------------------------
B1
B1
R1     2002
R48    2002
Name: count, dtype: int64
----------------------------------------
B4
B4
No    4004
Name: count, dtype: int64
----------------------------------------


In [75]:
# ==================================================
# 0. IMPORT LIBRARIES
# ==================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score,
    mean_absolute_error, mean_squared_error
)

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


# ==================================================
# 1. LOAD DATASET
# ==================================================
file_path = r"E:\WOU_BITS INTERN\39 BUS SYSTEM\39 BUS SYSTEM FAULT.xlsx"
df = pd.read_excel(file_path)

print("Original Dataset Shape:", df.shape)


# ==================================================
# 2. FILTER ONLY FAULT LINE 1_39
# ==================================================
df = df[df["Fault Line"] == "1_39"].copy()

print("Filtered Dataset Shape (Only 1_39):", df.shape)


# ==================================================
# 3. DEFINE FEATURES AND LABELS
# ==================================================
FEATURES = [
    "Phase Voltage A/Terminal in kV",
    "Phase Voltage B/Terminal in kV",
    "Phase Voltage C/Terminal in kV",
    "Phase Current A/Terminal in kA",
    "Phase Current B/Terminal in kA",
    "Phase Current C/Terminal in kA",
    "Terminal/Bus"
]

# 🔥 ONLY required labels (NO B4)
LABELS = [
    "Fault Location",
    "Primary",
    "CCT",
    "B1"
]

print("Labels being used:", LABELS)

X = df[FEATURES]

# ==================================================
# 4. FEATURE SCALING
# ==================================================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

final_summary = {}


# ==================================================
# 5. LOOP THROUGH EACH LABEL
# ==================================================
for label in LABELS:

    print("\n" + "=" * 60)
    print(f"🎯 Processing Label: {label}")
    print("=" * 60)

    # Skip if only one class
    if df[label].nunique() < 2:
        print(f"❌ {label} skipped — only one class present")
        continue

    # Encode target
    le = LabelEncoder()
    y = le.fit_transform(df[label])

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y,
        test_size=0.25,
        random_state=42,
        stratify=y
    )

    # ==================================================
    # RANDOM FOREST
    # ==================================================
    rf = RandomForestClassifier(
        n_estimators=300,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_prob = rf.predict_proba(X_test)

    acc_rf = accuracy_score(y_test, rf_pred) * 100


    # ==================================================
    # XGBOOST (Binary Safe Version)
    # ==================================================
    xgb = XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    xgb.fit(X_train, y_train)

    xgb_pred = xgb.predict(X_test)
    xgb_prob = xgb.predict_proba(X_test)

    acc_xgb = accuracy_score(y_test, xgb_pred) * 100


    # ==================================================
    # ENSEMBLE (Soft Voting)
    # ==================================================
    best_alpha = 0
    best_acc = 0
    best_pred = None
    best_prob = None

    for alpha in np.arange(0.1, 1.0, 0.1):
        ens_prob = alpha * rf_prob + (1 - alpha) * xgb_prob
        ens_pred = np.argmax(ens_prob, axis=1)
        acc = accuracy_score(y_test, ens_pred) * 100

        if acc > best_acc:
            best_acc = acc
            best_alpha = alpha
            best_pred = ens_pred
            best_prob = ens_prob


    # ==================================================
    # MODEL SELECTION
    # ==================================================
    scores = {
        "Random Forest": acc_rf,
        "XGBoost": acc_xgb,
        "Ensemble": best_acc
    }

    best_model = max(scores, key=scores.get)

    if best_model == "Random Forest":
        y_pred = rf_pred
        y_prob = rf_prob
        final_acc = acc_rf
    elif best_model == "XGBoost":
        y_pred = xgb_pred
        y_prob = xgb_prob
        final_acc = acc_xgb
    else:
        y_pred = best_pred
        y_prob = best_prob
        final_acc = best_acc


    # ==================================================
    # FINAL METRICS
    # ==================================================
    precision = precision_score(y_test, y_pred, average="weighted") * 100
    recall = recall_score(y_test, y_pred, average="weighted") * 100
    f1 = f1_score(y_test, y_pred, average="weighted") * 100
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # For binary ROC-AUC
    try:
        roc_auc = roc_auc_score(y_test, y_prob[:, 1]) * 100
    except:
        roc_auc = None


    # ==================================================
    # PRINT RESULTS
    # ==================================================
    print(f"RF Accuracy       : {acc_rf:.2f}%")
    print(f"XGB Accuracy      : {acc_xgb:.2f}%")
    print(f"Ensemble Accuracy : {best_acc:.2f}% (alpha={best_alpha})")
    print(f"✅ Selected Model  : {best_model}")

    print("\n--- FINAL METRICS ---")
    print(f"Accuracy  : {final_acc:.2f}%")
    print(f"Precision : {precision:.2f}%")
    print(f"Recall    : {recall:.2f}%")
    print(f"F1-score  : {f1:.2f}%")
    print(f"ROC-AUC   : {roc_auc:.2f}%" if roc_auc else "ROC-AUC : Not defined")
    print(f"MAE       : {mae:.4f}")
    print(f"RMSE      : {rmse:.4f}")

    final_summary[label] = {
        "Best Model": best_model,
        "Accuracy": final_acc
    }


# ==================================================
# FINAL SUMMARY
# ==================================================
print("\n🎯 FINAL SUMMARY (Fault Line 1_39)")
print("=" * 60)

for lbl, res in final_summary.items():
    print(f"{lbl:15s} → {res['Best Model']:15s} | Accuracy = {res['Accuracy']:.2f}%")

print("\n✅ PROCESS COMPLETED SUCCESSFULLY")

Original Dataset Shape: (135136, 24)
Filtered Dataset Shape (Only 1_39): (4004, 24)
Labels being used: ['Fault Location', 'Primary', 'CCT', 'B1']

🎯 Processing Label: Fault Location
RF Accuracy       : 99.70%
XGB Accuracy      : 99.70%
Ensemble Accuracy : 99.70% (alpha=0.1)
✅ Selected Model  : Random Forest

--- FINAL METRICS ---
Accuracy  : 99.70%
Precision : 99.70%
Recall    : 99.70%
F1-score  : 99.70%
ROC-AUC   : 100.00%
MAE       : 0.0030
RMSE      : 0.0547

🎯 Processing Label: Primary
RF Accuracy       : 100.00%
XGB Accuracy      : 100.00%
Ensemble Accuracy : 100.00% (alpha=0.1)
✅ Selected Model  : Random Forest

--- FINAL METRICS ---
Accuracy  : 100.00%
Precision : 100.00%
Recall    : 100.00%
F1-score  : 100.00%
ROC-AUC   : 100.00%
MAE       : 0.0000
RMSE      : 0.0000

🎯 Processing Label: CCT
RF Accuracy       : 99.90%
XGB Accuracy      : 99.90%
Ensemble Accuracy : 99.90% (alpha=0.1)
✅ Selected Model  : Random Forest

--- FINAL METRICS ---
Accuracy  : 99.90%
Precision : 99.90%
R

In [85]:
# ==================================================
# 0. IMPORT LIBRARIES
# ==================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score,
    mean_absolute_error, mean_squared_error
)

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


# ==================================================
# 1. LOAD DATASET
# ==================================================
file_path = r"E:\WOU_BITS INTERN\39 BUS SYSTEM\39 BUS SYSTEM FAULT.xlsx"
df = pd.read_excel(file_path)

print("Original Dataset Shape:", df.shape)


# ==================================================
# 2. FILTER ONLY FAULT LINE 1_39
# ==================================================
df = df[df["Fault Line"] == "1_39"].copy()

print("Filtered Dataset Shape (Only 1_39):", df.shape)


# ==================================================
# 3. DEFINE FEATURES AND LABELS
# ==================================================
FEATURES = [
    "Phase Voltage A/Terminal in kV",
    "Phase Voltage B/Terminal in kV",
    "Phase Voltage C/Terminal in kV",
    "Phase Current A/Terminal in kA",
    "Phase Current B/Terminal in kA",
    "Phase Current C/Terminal in kA",
    "Terminal/Bus"
]

# 🔥 ONLY required labels (NO B4)
LABELS = [
    "Fault Location",
    "Primary",
    "CCT",
    "B1"
]

print("Labels being used:", LABELS)

X = df[FEATURES]

# ==================================================
# 4. FEATURE SCALING
# ==================================================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

final_summary = {}


# ==================================================
# 5. LOOP THROUGH EACH LABEL
# ==================================================
for label in LABELS:

    print("\n" + "=" * 60)
    print(f"🎯 Processing Label: {label}")
    print("=" * 60)

    # Skip if only one class
    if df[label].nunique() < 2:
        print(f"❌ {label} skipped — only one class present")
        continue

    # Encode target
    le = LabelEncoder()
    y = le.fit_transform(df[label])

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y,
        test_size=0.25,
        random_state=42,
        stratify=y
    )

    # ==================================================
    # RANDOM FOREST
    # ==================================================
    rf = RandomForestClassifier(
        n_estimators=250,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_prob = rf.predict_proba(X_test)

    acc_rf = accuracy_score(y_test, rf_pred) * 100


    # ==================================================
    # XGBOOST (Binary Safe Version)
    # ==================================================
    xgb = XGBClassifier(
        n_estimators=350,
        learning_rate=0.07,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    xgb.fit(X_train, y_train)

    xgb_pred = xgb.predict(X_test)
    xgb_prob = xgb.predict_proba(X_test)

    acc_xgb = accuracy_score(y_test, xgb_pred) * 100


    # ==================================================
    # ENSEMBLE (Soft Voting)
    # ==================================================
    best_alpha = 0
    best_acc = 0
    best_pred = None
    best_prob = None

    for alpha in np.arange(0.1, 1.0, 0.1):
        ens_prob = alpha * rf_prob + (1 - alpha) * xgb_prob
        ens_pred = np.argmax(ens_prob, axis=1)
        acc = accuracy_score(y_test, ens_pred) * 100

        if acc > best_acc:
            best_acc = acc
            best_alpha = alpha
            best_pred = ens_pred
            best_prob = ens_prob


    # ==================================================
    # MODEL SELECTION
    # ==================================================
    scores = {
        "Random Forest": acc_rf,
        "XGBoost": acc_xgb,
        "Ensemble": best_acc
    }

    best_model = max(scores, key=scores.get)

    if best_model == "Random Forest":
        y_pred = rf_pred
        y_prob = rf_prob
        final_acc = acc_rf
    elif best_model == "XGBoost":
        y_pred = xgb_pred
        y_prob = xgb_prob
        final_acc = acc_xgb
    else:
        y_pred = best_pred
        y_prob = best_prob
        final_acc = best_acc


    # ==================================================
    # FINAL METRICS
    # ==================================================
    precision = precision_score(y_test, y_pred, average="weighted") * 100
    recall = recall_score(y_test, y_pred, average="weighted") * 100
    f1 = f1_score(y_test, y_pred, average="weighted") * 100
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # For binary ROC-AUC
    try:
        roc_auc = roc_auc_score(y_test, y_prob[:, 1]) * 100
    except:
        roc_auc = None


    # ==================================================
    # PRINT RESULTS
    # ==================================================
    print(f"RF Accuracy       : {acc_rf:.2f}%")
    print(f"XGB Accuracy      : {acc_xgb:.2f}%")
    print(f"Ensemble Accuracy : {best_acc:.2f}% (alpha={best_alpha})")
    print(f"✅ Selected Model  : {best_model}")

    print("\n--- FINAL METRICS ---")
    print(f"Accuracy  : {final_acc:.2f}%")
    print(f"Precision : {precision:.2f}%")
    print(f"Recall    : {recall:.2f}%")
    print(f"F1-score  : {f1:.2f}%")
    print(f"ROC-AUC   : {roc_auc:.2f}%" if roc_auc else "ROC-AUC : Not defined")
    print(f"MAE       : {mae:.4f}")
    print(f"RMSE      : {rmse:.4f}")

    final_summary[label] = {
        "Best Model": best_model,
        "Accuracy": final_acc
    }


# ==================================================
# FINAL SUMMARY
# ==================================================
print("\n🎯 FINAL SUMMARY (Fault Line 1_39)")
print("=" * 60)

for lbl, res in final_summary.items():
    print(f"{lbl:15s} → {res['Best Model']:15s} | Accuracy = {res['Accuracy']:.2f}%")

print("\n✅ PROCESS COMPLETED SUCCESSFULLY")

Original Dataset Shape: (135136, 24)
Filtered Dataset Shape (Only 1_39): (4004, 24)
Labels being used: ['Fault Location', 'Primary', 'CCT', 'B1']

🎯 Processing Label: Fault Location
RF Accuracy       : 99.70%
XGB Accuracy      : 99.70%
Ensemble Accuracy : 99.70% (alpha=0.1)
✅ Selected Model  : Random Forest

--- FINAL METRICS ---
Accuracy  : 99.70%
Precision : 99.70%
Recall    : 99.70%
F1-score  : 99.70%
ROC-AUC   : 100.00%
MAE       : 0.0030
RMSE      : 0.0547

🎯 Processing Label: Primary
RF Accuracy       : 100.00%
XGB Accuracy      : 100.00%
Ensemble Accuracy : 100.00% (alpha=0.1)
✅ Selected Model  : Random Forest

--- FINAL METRICS ---
Accuracy  : 100.00%
Precision : 100.00%
Recall    : 100.00%
F1-score  : 100.00%
ROC-AUC   : 100.00%
MAE       : 0.0000
RMSE      : 0.0000

🎯 Processing Label: CCT
RF Accuracy       : 99.90%
XGB Accuracy      : 99.90%
Ensemble Accuracy : 99.90% (alpha=0.1)
✅ Selected Model  : Random Forest

--- FINAL METRICS ---
Accuracy  : 99.90%
Precision : 99.90%
R